In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import os, sys
from functools import reduce
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

C:\Users\keypg\Anaconda3\envs\ml\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\keypg\Anaconda3\envs\ml\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\keypg\Anaconda3\envs\ml\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:\n%s" %


In [2]:
df = pd.read_csv('./dataset/all_data.csv')
df.head(n=5)

,Ticker,Date,Open,High,Low,Close,Adj Close,Volume
0,AAPL,2018-11-20,44.592499,45.367500,43.877499,44.244999,42.865883,271300800
1,AAPL,2018-11-21,44.932499,45.067501,44.137501,44.195000,42.817429,124496800
2,AAPL,2018-11-23,43.735001,44.150002,43.025002,43.072498,41.729916,94496000
3,AAPL,2018-11-26,43.560001,43.737499,42.564999,43.654999,42.294266,179994000
4,AAPL,2018-11-27,42.877499,43.692501,42.720001,43.560001,42.202229,165549600


In [3]:
stocks = df.drop(columns=['Date', 'Adj Close'])
stocks = stocks.groupby(['Ticker'])
stocks.count()

,Open,High,Low,Close,Volume
Ticker,,,,,
AAPL,510,510,510,510,510
AMZN,510,510,510,510,510
EBAY,510,510,510,510,510
FB,510,510,510,510,510
GOOG,510,510,510,510,510
MSFT,510,510,510,510,510
NFLX,510,510,510,510,510
NKE,510,510,510,510,510
NVDA,510,510,510,510,510


In [4]:
print(df.shape, stocks.get_group('AAPL').shape)

(5100, 8) (510, 6)


In [5]:
stock_name = []
stocks_df_list = []
for name, group in stocks:
    stock_name.append(name)
    stocks_df_list.append(group.drop(columns=['Ticker']).div([100, 100, 100, 100, 100000]).to_numpy()[:504, :].reshape((72, 7, 5))) # divide for normalize dataset * don't forgot to divide the input when use with real data
    print(f'stock: {name}\tshape : {stocks_df_list[-1].shape}')

stock: AAPL	shape : (72, 7, 5)
stock: AMZN	shape : (72, 7, 5)
stock: EBAY	shape : (72, 7, 5)
stock: FB	shape : (72, 7, 5)
stock: GOOG	shape : (72, 7, 5)
stock: MSFT	shape : (72, 7, 5)
stock: NFLX	shape : (72, 7, 5)
stock: NKE	shape : (72, 7, 5)
stock: NVDA	shape : (72, 7, 5)
stock: TSLA	shape : (72, 7, 5)


In [6]:
reduce_stocks = reduce(lambda a,b: np.concatenate((a, b)), stocks_df_list) # trim total samples in each stock into 504 to make it can devide into set of 7
features, labels = reduce_stocks[0::2], reduce_stocks[1::2][:,:,3]*100 # label is a close price, multiply by 100 to make it back to original range

In [7]:
print(f'features shape: {features.shape}\nlabels shape: {labels.shape}')
print(reduce_stocks.shape)

features shape: (360, 7, 5)
labels shape: (360, 7)
(720, 7, 5)


In [8]:
print(features[0], features[0].shape)
print('\n\n', labels[0], labels[0].shape)

[[4.45924988e-01 4.53675003e-01 4.38774986e-01 4.42449989e-01
  2.71300800e+03]
 [4.49324989e-01 4.50675011e-01 4.41375008e-01 4.41949997e-01
  1.24496800e+03]
 [4.37350006e-01 4.41500015e-01 4.30250015e-01 4.30724983e-01
  9.44960000e+02]
 [4.35600014e-01 4.37374992e-01 4.25649986e-01 4.36549988e-01
  1.79994000e+03]
 [4.28774986e-01 4.36925011e-01 4.27200012e-01 4.35600014e-01
  1.65549600e+03]
 [4.41824989e-01 4.53224983e-01 4.37324982e-01 4.52350006e-01
  1.84250000e+03]
 [4.56650009e-01 4.57000008e-01 4.44249992e-01 4.48875008e-01
  1.67080000e+03]] (7, 5)


 [44.64500046 46.20500183 44.17250061 43.68000031 42.12250137 42.40000153
 42.15750122] (7,)


In [9]:
# save several arrays into compressed .npz archive
with open('dataset/stock_dataset_preprocessed.npz', 'wb') as f:
    np.savez(f, features=features, labels=labels)